In [ ]:
!git clone -b master https://github.com/charles9n/bert-sklearn
!cd bert-sklearn; pip install .

Cloning into 'bert-sklearn'...
remote: Enumerating objects: 259, done.
remote: Total 259 (delta 0), reused 0 (delta 0), pack-reused 259
Receiving objects: 100% (259/259), 516.15 KiB | 12.90 MiB/s, done.
Resolving deltas: 100% (131/131), done.
Processing /content/bert-sklearn
     |████████████████████████████████| 133kB 7.6MB/s 
     |████████████████████████████████| 81kB 9.5MB/s 
     |████████████████████████████████| 7.7MB 48.0MB/s 
  Created wheel for bert-sklearn: filename=bert_sklearn-0.3.1-cp37-none-any.whl size=54248 sha256=9093ecbffcf643f71ac7ae6910f72f51d895d8e2581504ff0aae10ece0119a3e
  Stored in directory: /root/.cache/pip/wheels/61/95/c6/5790aae8fb377f5ff356dbe58205aab28858595d6bff8197d0
Successfully built bert-sklearn
ERROR: botocore 1.20.109 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from bert_sklearn import BertClassifier

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/aatishsuman/velocityEHS-system-design/master/review.csv')
df

,Condition Number,Condition Text,Classification,Requirement Description,Requirement Name,Referable Conditions
0,1.,Point of emissions and applicable emission lim...,DSC,NaN,NaN,NaN
1,1.1.,EUG 1 Plant-wide This EUG is established to ad...,DSC,NaN,NaN,NaN
2,1.2.,EUG 2 Package Boilers,DSC,,NaN,NaN
3,1.2.A.,Fuel usage of the boilers shall not exceed a t...,REQ,Fuel usage of the boilers shall not exceed a t...,1.2.A.,"1., 1.2., 1.2.A."
4,1.2.B.,These boilers are affected facilities under 40...,REQ,These boilers are affected facilities under 40...,1.2.B.,"1., 1.2., 1.2.B."
...,...,...,...,...,...,...
91,1.8.A.,The permittee shall monitor and record natural...,REQ,The permittee shall monitor and record natural...,1.8.A.,"1., 1.8., 1.8.A."
92,1.8.B.,Plant #2 is subject to the emission limitation...,DSC,NaN,NaN,NaN
93,1.8.C.,Reference Method performance testing of all po...,REQ,Reference Method performance testing of all po...,1.8.C.,"1., 1.8., 1.8.C."
94,1.9.,EUG 9 Ammonia Plant Start-up and Shutdown Vent...,REQ,EUG 9 Ammonia Plant Start-up and Shutdown Vent...,1.9.,"1., 1.9."


In [ ]:
# splitting the data into train and test splits (0.75 / 0.25)
X_train, X_test, y_train, y_test = train_test_split(df['Condition Text'], df['Classification'], test_size=0.25, random_state=42)
# Printing the distribution across the classes for all, train and test data
print(df['Classification'].value_counts())
print(y_train.value_counts())
print(y_test.value_counts())

REQ    61
DSC    32
RAE     3
Name: Classification, dtype: int64
REQ    43
DSC    28
RAE     1
Name: Classification, dtype: int64
REQ    18
DSC     4
RAE     2
Name: Classification, dtype: int64


In [ ]:
# trying the BioBERT-based model as the dataset contains technical langauge
model = BertClassifier(bert_model='biobert-base-cased', random_state=42, validation_fraction=0)
print(model)
model.fit(X_train, y_train)

Building sklearn text classifier...
BertClassifier(bert_config_json=None, bert_model='biobert-base-cased',
               bert_vocab=None, do_lower_case=None, epochs=3, eval_batch_size=8,
               fp16=False, from_tf=False, gradient_accumulation_steps=1,
               ignore_label=None, label_list=None, learning_rate=2e-05,
               local_rank=-1, logfile='bert_sklearn.log', loss_scale=0,
               max_seq_length=128, num_mlp_hiddens=500, num_mlp_layers=0,
               random_state=42, restore_file=None, train_batch_size=32,
               use_cuda=True, validation_fraction=0, warmup_proportion=0.1)
Loading biobert-base-cased model...
Defaulting to linear classifier/regressor
Loading Tensorflow checkpoint from  model.ckpt-1000000

train data size: 72, validation data size: 0



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


BertClassifier(bert_config_json=None, bert_model='biobert-base-cased',
               bert_vocab=None, do_lower_case=False, epochs=3,
               eval_batch_size=8, fp16=False, from_tf=False,
               gradient_accumulation_steps=1, ignore_label=None,
               label_list=array(['DSC', 'RAE', 'REQ'], dtype=object),
               learning_rate=2e-05, local_rank=-1, logfile='bert_sklearn.log',
               loss_scale=0, max_seq_length=128, num_mlp_hiddens=500,
               num_mlp_layers=0, random_state=42, restore_file=None,
               train_batch_size=32, use_cuda=True, validation_fraction=0,
               warmup_proportion=0.1)

In [ ]:
# training data results
X_train_pred = model.predict(X_train)
print('Train classification report:\n', classification_report(y_train, X_train_pred, target_names=['DSC', 'RAE', 'REQ']), '\n')
print('Confusion matrix -\n', confusion_matrix(y_train, X_train_pred, labels=['DSC', 'RAE', 'REQ']))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



Train classification report:
               precision    recall  f1-score   support

         DSC       1.00      0.86      0.92        28
         RAE       0.00      0.00      0.00         1
         REQ       0.90      1.00      0.95        43

    accuracy                           0.93        72
   macro avg       0.63      0.62      0.62        72
weighted avg       0.92      0.93      0.92        72
 

Confusion matrix -
 [[24  0  4]
 [ 0  0  1]
 [ 0  0 43]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# test data results
X_test_pred = model.predict(X_test)
print('Test classification report:\n', classification_report(y_test, X_test_pred, target_names=['DSC', 'RAE', 'REQ']), '\n')
print('Confusion matrix -\n', confusion_matrix(y_test, X_test_pred, labels=['DSC', 'RAE', 'REQ']))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



Test classification report:
               precision    recall  f1-score   support

         DSC       1.00      0.75      0.86         4
         RAE       0.00      0.00      0.00         2
         REQ       0.86      1.00      0.92        18

    accuracy                           0.88        24
   macro avg       0.62      0.58      0.59        24
weighted avg       0.81      0.88      0.84        24
 

Confusion matrix -
 [[ 3  0  1]
 [ 0  0  2]
 [ 0  0 18]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# DSC predicted as REQ in training data
list(X_train[(y_train == 'DSC') & (X_train_pred == 'REQ')])

['Definitions.  Terms used in this CEMS Plan that are defined in the Clean Air Act (CAA) or in federal or state regulations promulgated pursuant to the CAA shall have the meaning assigned to them in the CAA or such regulations, unless otherwise defined in the CD. The terms used in this CEMS Plan that are defined in the CD shall have the meaning assigned to them therein. The following definitions specifically apply for purposes of this CEMS Plan.',
 'Monitoring of fuel sulfur content is not required when a gaseous fuel is fired in the turbine and the owner or operator demonstrates that the gaseous fuel meets the definition of "natural gas" using one of the methods in $60.334(h)(3)(i) or (ii). Paragraph 60.331 defines natural gas as containing 20 grains or less of total sulfur per 100 standard cubic feet and is either composed of at least 70 percent methane by volume or has a gross caloric value between 950 and 1100 BTU/scf. [40 CFR60.334(h)(3)]',
 'Principles.  This CEMS Plan is the mec

In [ ]:
# DSC predicted as REQ in test data
list(X_test[(y_test == 'DSC') & (X_test_pred == 'REQ')])

['The CD established a CEMS Monitoring Plan that consists of sections titled: Principles; Definitions; Emissions Monitoring; Production Data; Conversion Factor; Emissions Calculations; Rounding of Numbers resulting from Calculations; Compliance with Consent Decree NOx Limits; Retention of All CEMS Data, including Data during Startup, Shutdown, and Malfunction; Analyzer Specifications; and Compliance with NSPS 40 CFR60, Subpart G. Each of these sections is described in the following paragraphs.']

## Error analysis

The errors occur mostly due the small size of the dataset. <br/>

For RAE classified as REQ, presence of linguistic cues like 'shall not', 'will be' and 'shall be', which are characteristic of the REQ class, causes the errors. The RAE instances can be differentiated from the REQ ones by the presence of other linguistic cues like 'following', which the model can learn with more data but 3 instances are not enough to identify this pattern. <br/>

For DSC classified as REQ also, the presence of linguistic cues like 'shall have', 'subject to', 'will', etc. causes the errors. There are other significant differences between the contents of the 2 classes which the model should be able to learn with more data.  

In [ ]:
# replacing NA values with ''
df.fillna(value='', inplace=True)

In [ ]:
# given a row, returns the Referable Conditions for the row
def getReferableConditions(row):
  conditions = []
  if row['Classification'] == 'DSC':
    return ''
  if row['Classification'] == 'REQ' or row['Classification'] == 'RAE':
    index_list = row['Condition Number'].split('.')
    for i in range(1, len(index_list)):
      conditions.append('.'.join(index_list[0:i]) + '.')
  if row['Classification'] == 'RAE':
    conditions += list(df['Condition Number'][row['Condition Number'] == df['Condition Number'].astype(str).apply(lambda e: '.'.join(e.split('.')[:-2]) + '.')])
  return ', '.join(conditions)

In [ ]:
df[df.apply(getReferableConditions, axis=1) != df['Referable Conditions']]

,Condition Number,Condition Text,Classification,Requirement Description,Requirement Name,Referable Conditions,prediction


*There are no instances where the above function does not produce the same value as in the Referable Conditions column.*

In [ ]:
# given a row, returns the Requirement Description for the row
def getRequirementDescription(row):
  if row['Classification'] != 'REQ':
    return ''
  rae_row = df[(df['Condition Number'] == '.'.join(row['Condition Number'].split('.')[:-2]) + '.') & (df['Classification'] == 'RAE')]
  if len(rae_row) > 0:
    return rae_row['Condition Number'].values[0] + '  ' + rae_row['Condition Text'].values[0] + '\n\n' + row['Condition Number'] + '  ' + row['Condition Text']
  return row['Condition Text']

In [ ]:
df[df.apply(getRequirementDescription, axis=1) != df['Requirement Description']]

,Condition Number,Condition Text,Classification,Requirement Description,Requirement Name,Referable Conditions,prediction
2,1.2.,EUG 2 Package Boilers,DSC,,,,DSC
69,1.6.H.,Production Data. Following each calendar day ...,REQ,Production Data Following each calendar day at...,1.6.H.,"1., 1.6., 1.6.H.",REQ
70,1.6.I.,Conversion Factor. During each required perfo...,REQ,Conversion Factor During each required perform...,1.6.I.,"1., 1.6., 1.6.I.",REQ
73,1.6.J.2.,Rolling 365-day average. Compliance with the ...,REQ,Rolling 365-day average. Compliance with the L...,1.6.J.2.,"1., 1.6., 1.6.J., 1.6.J.2.",REQ


*There are 4 instances where the value returned by the above function is different from that in the Requirement Description column. For the first one, the value in the column is a blank space, for the next 2, the period is missing in Requirement Description column, and for the last one, there is an additional space after the period in the first line in the Condition Text.*